In [ ]:
import pandas as pd
import numpy as np

# Definindo os valores possíveis para cada coluna
generos = ['Ação', 'Comédia', 'Drama', 'Animação', 'Ficção Científica']
idades = np.random.randint(15, 81, size=10000) # Valores aleatórios entre 16 e 80 anos
orcamentos = np.random.randint(50, 500, size=10000) * 1000  # Valores em milhares de dólares
alcance_previsto = np.random.randint(10000, 100000, size=10000)
cpcs = np.random.uniform(0.3, 0.7, size=10000).round(2)  # Valores aleatórios entre 0.3 e 0.7
conversoes = np.random.randint(5000, 500000, size=10000)

# Calculando o ROI com base no orçamento da campanha
roi = orcamentos * 2.87

# Função para definir a faixa etária com base na idade
def definir_faixa_etaria(idade):
    if idade <= 25:
        return '16-25'
    elif idade <= 35:
        return '26-35'
    elif idade <= 50:
        return '36-50'
    elif idade <= 70:
        return '51-70'
    else:
        return 'Mais de 70'

# Aplicando a função para definir a faixa etária
faixas_etarias = [definir_faixa_etaria(idade) for idade in idades]

# Função para selecionar o canal de marketing com base na faixa etária
def selecionar_canal(faixa_etaria):
    if faixa_etaria == '16-25':
        return 'Twitter e TikTok'
    elif faixa_etaria == '26-35':
        return 'Facebook e Instagram'
    elif faixa_etaria == '36-50':
        return 'Google Ads e YouTube'
    elif faixa_etaria == '51-70':
        return 'e-mail'
    else:
        return 'Outros'

# Aplicando a função para definir os canais de marketing
canais_marketing = [selecionar_canal(faixa_etaria) for faixa_etaria in faixas_etarias]

# Criando o DataFrame
df = pd.DataFrame({
    'Gênero do Filme': np.random.choice(generos, size=10000),
    'Faixa Etária': faixas_etarias,
    'Orçamento da Campanha': orcamentos,
    'Alcance Previsto': alcance_previsto,
    'ROI': roi,
    'Canais de Marketing': canais_marketing,
    'CPC Médio (em dólares)': cpcs,
    'Conversões': conversoes
})

# Visualizando o DataFrame
display(df)


,Gênero do Filme,Faixa Etária,Orçamento da Campanha,Alcance Previsto,ROI,Canais de Marketing,CPC Médio (em dólares),Conversões
0,Comédia,36-50,239000,79047,685930.0,Google Ads e YouTube,0.66,284994
1,Drama,51-70,331000,23016,949970.0,e-mail,0.32,135293
2,Comédia,16-25,412000,87244,1182440.0,Twitter e TikTok,0.35,482015
3,Drama,Mais de 70,152000,89089,436240.0,Outros,0.62,36426
4,Ficção Científica,26-35,363000,98276,1041810.0,Facebook e Instagram,0.46,5230
...,...,...,...,...,...,...,...,...
9995,Comédia,Mais de 70,207000,83799,594090.0,Outros,0.55,40467
9996,Ação,36-50,351000,56127,1007370.0,Google Ads e YouTube,0.50,97931
9997,Comédia,Mais de 70,417000,20057,1196790.0,Outros,0.36,312771
9998,Ação,51-70,98000,15967,281260.0,e-mail,0.61,387272
